# Import a Triangle Mesh Geoscience Object from an OBJ file

This notebook shows how to load and import a Triangle Mesh Geoscience Object from an OBJ file.

To connect with Evo, credentials to login need to be provided, by filling in the environment variables
`EVO_CLIENT_ID`, `EVO_BASE_URI` and `EVO_DISCOVERY_URL` and loading them into your Jupyter environment.

Once logged in, a widget will be displayed below allowing you to select an organisation and workspace to publish objects to.

You can then pick from the OBJ files that are placed in the `data` subdirectory of where this notebook is and choose to upload it.

There are multiple importer implementations supported, the recommended one is the default "trimesh", but "tinyobj" can also be
used to process larger meshes.

In [ ]:
import os
from pprint import pp

from evo.data_converters.common import create_evo_object_service_and_data_client
from evo.notebooks import ServiceManagerWidget

client_id = os.getenv("EVO_CLIENT_ID", "")
base_uri = os.getenv("EVO_BASE_URI", "")
discovery_url = os.getenv("EVO_DISCOVERY_URL", "")

manager = await ServiceManagerWidget.with_auth_code(
    client_id=client_id, base_uri=base_uri, discovery_url=discovery_url
).login()

object_service_client, data_client = create_evo_object_service_and_data_client(service_manager_widget=manager)

In [ ]:
from glob import glob
from pathlib import Path

import ipywidgets as widgets

data_dir = Path.cwd() / "data"
obj_files = [p for p in glob("**/*.obj", recursive=True, root_dir=data_dir)]

file_selector = widgets.RadioButtons(options=obj_files, disabled=False, layout={"width": "max-content"})

display(file_selector)

In [ ]:
from evo.data_converters.obj.importer.obj_to_evo import convert_obj

test_file = data_dir / file_selector.value
assert test_file.exists()

tags = {"TagName": "Tag value"}
implementation = "trimesh"
objects_metadata = await convert_obj(
    filepath=test_file,
    implementation=implementation,
    epsg_code=None,
    service_manager_widget=manager,
    tags=tags,
    upload_path=f"test-obj-{implementation}",
    publish_objects=True,
    overwrite_existing_objects=True,
)

print()
print("These objects have now been published:")

for metadata in objects_metadata:
    pp(metadata, indent=4)